In [1]:
import os
from os import listdir
from PIL import Image
from numpy import asarray
from numpy import expand_dims
from matplotlib import pyplot
from keras.models import load_model
from keras_facenet import FaceNet
import numpy as np

import pickle
import cv2

In [2]:
#HaarCascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
HaarCascade = cv2.CascadeClassifier(cv2.samples.findFile(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml'))
MyFaceNet = FaceNet()

In [3]:
# Directory containing folders of images for each person
folder = 'images/'

# Initialize database dictionary
database = {}

# Function to extract embeddings from images
def extract_embeddings(image_path):
    # Read image from file
    gbr1 = cv2.imread(image_path)
    
    # Detect face
    haar = HaarCascade.detectMultiScale(gbr1, 1.1, 4)
    
    if len(haar) > 0:
        x1, y1, width, height = haar[0]
    else:
        return None  # Skip images without faces
    
    x1, y1 = abs(x1), abs(y1)
    x2, y2 = x1 + width, y1 + height
    
    # Convert image to RGB and resize for FaceNet
    gbr = cv2.cvtColor(gbr1, cv2.COLOR_BGR2RGB)
    gbr = Image.fromarray(gbr)
    gbr_array = asarray(gbr)
    face = gbr_array[y1:y2, x1:x2]
    face = Image.fromarray(face)
    face = face.resize((160, 160))
    face = asarray(face)
    
    # Expand dimensions for FaceNet input
    face = expand_dims(face, axis=0)
    
    # Get FaceNet embedding
    signature = MyFaceNet.embeddings(face)
    
    return signature

# Traverse through each folder in 'images/' and use folder names as labels
for person_folder in os.listdir(folder):
    person_path = os.path.join(folder, person_folder)
    if os.path.isdir(person_path):
        print(f"Processing images for {person_folder}")
        
        # Initialize lists to store embeddings and weights for the person
        embeddings = []
        weights = []
        
        # Traverse through images in the person's folder
        for filename in os.listdir(person_path):
            image_path = os.path.join(person_path, filename)
            
            # Extract embedding from image if face is detected
            signature = extract_embeddings(image_path)
            if signature is not None:
                embeddings.append(signature)
                weights.append(1.0)  # Default weight for each embedding
        
        # Calculate weighted average embedding for the person
        if embeddings:
            weighted_avg_embedding = np.average(embeddings, axis=0, weights=weights)
            
            # Add person's weighted average embedding to the database
            database[person_folder] = weighted_avg_embedding

print("Embeddings extraction completed.")

Processing images for Sudharsan
1/1 [==============================] - 0s 38ms/step
Processing images for Sukash
1/1 [==============================] - 0s 41ms/step
Processing images for Vignesh
1/1 [==============================] - 0s 37ms/step
Embeddings extraction completed.


In [4]:
myfile = open("data.pkl", "wb")
pickle.dump(database, myfile)
myfile.close()

In [5]:
database

{'Sudharsan': array([[ 3.58500869e-02, -5.81452104e-02,  9.34413619e-03,
          4.20041685e-02, -5.63888729e-03,  2.24830819e-02,
          9.45373594e-03,  6.82866736e-03, -4.76188736e-02,
         -1.94346411e-02,  1.05383533e-02, -2.05858029e-02,
          1.01846415e-02,  4.41234902e-02,  3.82800083e-03,
         -2.21479147e-03,  5.96666857e-03,  7.68833917e-02,
          1.50057180e-02,  1.84330474e-02, -2.55650186e-02,
          3.00153110e-02,  5.74721708e-02, -6.10002554e-02,
         -1.69688531e-02,  5.69931603e-03,  2.40453368e-02,
         -3.46056783e-02,  4.29083578e-02, -5.61333318e-02,
         -1.66736422e-02,  4.43654496e-02, -7.99114771e-03,
          4.42378088e-03, -1.51063772e-02, -6.60506946e-04,
          6.72045294e-03,  2.97383866e-02, -1.81203369e-02,
         -2.74958711e-02, -1.12058463e-03, -9.62652127e-03,
         -6.01580089e-03, -1.63363591e-02, -7.75378083e-04,
         -5.50000776e-02,  2.28256189e-03,  3.57966320e-02,
         -7.38634577e-02,  